In [1]:
import tensorflow as tf
import tf_keras
import tensorflow_model_optimization as tfmot
from tf_keras import layers,losses,optimizers,Sequential
from tf_keras.models import Model
from tf_keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D,Input
from tf_keras.applications import VGG16
from tf_keras import datasets,models
import importlib

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
from importing_dataset import load_dataset
import APoZ
importlib.reload(APoZ)
from APoZ import APoZ_Algorithm
train_examples, validation_examples, num_examples, num_classes, class_names = load_dataset('horses_or_humans', 70)

In [4]:
from reformatting import reformat_image
BATCH_SIZE = 32
IMG_SIZE = (224, 224)
train_batches = train_examples.cache().shuffle(num_examples//4).map(reformat_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(reformat_image).batch(BATCH_SIZE).prefetch(1)
train_batches

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [6]:
# # Chargez le modèle
# APoZ_model = models.load_model("model_experiments/horses_or_humans_ApoZ.keras")
# APoZ_model.summary()
#
#
# # Crée un modèle d'activation pour les couches convolutives du VGG16
# activation_model = Model(
#     inputs=APoZ_model.input,
#     outputs=[layer.output for layer in APoZ_model.layers[1].layers if isinstance(layer, layers.Conv2D)]  # Accéder directement aux couches internes du VGG16
# )
# activation_model.summary()
# activation_model.summary()

In [9]:
# # Chargez le modèle complet (assurez-vous que le modèle est bien formé et connecté)
# APoZ_model = models.load_model("model_experiments/horses_or_humans_ApoZ.keras")
# APoZ_model.summary()
#
# # Liste des couches Conv2D
# conv_layers = [layer.output for layer in APoZ_model.layers if isinstance(layer, layers.Conv2D)]
#
# # Crée un nouveau modèle qui prend les entrées du modèle original et produit les sorties des couches Conv2D
# activation_model = Model(inputs=APoZ_model.input, outputs=conv_layers)
#
# # Résumé du modèle d'activations
# activation_model.summary()
#
# # # Obtenir les activations avec un jeu de données (exemple ici avec des données de validation)
# # activations = activation_model.predict(validation_data)  # Remplace validation_data par tes données
# #
# # # Vérification des activations obtenues
# # print("Activations:", activations)


In [10]:
# # Charger le modèle VGG16 sans les couches Fully Connected (include_top=False)
# vgg16_model = VGG16(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
#
# # Afficher la structure du modèle pour comprendre quelles couches sont disponibles
# vgg16_model.summary()
# vgg16_model.compile(
#     optimizer = 'adam',
#     loss = losses.SparseCategoricalCrossentropy(),
#     metrics=['accuracy'])
#
# EPOCHS = 3
# history = vgg16_model.fit(train_batches,
#                     epochs = EPOCHS,
#                     batch_size = BATCH_SIZE,
#                     validation_data = validation_batches)
#
# # Liste des couches Conv2D pour lesquelles on veut obtenir les activations
# conv_layers = [layer.output for layer in vgg16_model.layers if isinstance(layer,
#                                                                           layers.Conv2D)]
#
# # Créer un modèle qui prend l'entrée de VGG16 et retourne les activations des couches Conv2D
# activation_model = Model(inputs=vgg16_model.input, outputs=conv_layers)
#
# # Afficher le résumé du modèle d'activations pour vérifier les sorties
# activation_model.summary()
#
# # Entrainer le modèle avec un jeu de données pour obtenir les activations
#
#
# # Charger un jeu de données d'images (par exemple une image unique ou un lot d'images)
# # Remplace 'test_data' par tes propres données d'entrée de forme (batch_size, 224, 224, 3)
# test_data = tf.random.normal((1, 224, 224, 3))  # Exemple d'image d'entrée aléatoire
#
# # Obtenir les activations des couches Conv2D pour l'image
# activations = activation_model.predict(test_data)
#
# # Afficher les dimensions des activations de chaque couche
# for i, activation in enumerate(activations):
#     print(f"Activations de la couche {i+1} : {activation.shape}")


In [11]:
# # Chargez le modèle
# feature_extractor = VGG16(input_shape=(IMG_SIZE + (3,)), include_top=False)
# feature_extractor.trainable = False
#
# max_pool_layer = layers.GlobalMaxPooling2D()
# prediction_layer = layers.Dense(num_classes, activation="softmax")
# inputs = Input(shape=IMG_SIZE + (3,))
# x = feature_extractor(inputs)
# x = max_pool_layer(x)
# x = Dropout(0.3)(x)
# outputs = prediction_layer(x)
# model = Model(inputs, outputs, name="horses_or_humans_ApoZ")
#
# model.summary()

In [12]:
# # Initialisation du feature extractor (VGG16)
# feature_extractor = VGG16(include_top=False, weights="imagenet", input_shape=(224, 224, 3))
#
# feature_extractor.summary()
# # Construire le modèle global
# max_pool_layer = layers.GlobalMaxPooling2D()
# prediction_layer = layers.Dense(num_classes, activation="softmax")
# inputs = Input(shape=(224, 224, 3))
# x = feature_extractor(inputs)  # Passer par VGG16
# x = max_pool_layer(x)
# x = Dropout(0.3)(x)
# outputs = prediction_layer(x)
# model = Model(inputs, outputs, name="horses_or_humans_ApoZ")
#
# # Afficher la structure complète
# model.summary()
#
# # Étape clé : Créer un modèle pour accéder aux activations des couches de VGG16
# conv_layers = [layer.output for layer in feature_extractor.layers if isinstance(layer, layers.Conv2D)]
# activation_model = Model(inputs=feature_extractor.input, outputs=conv_layers)
#
# activation_model.summary()


In [5]:
# Initialiser les couches une par une
inputs = Input(shape=(224, 224, 3))

# Bloc 1
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(inputs)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

# Bloc 2
x = layers.Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
x = layers.Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

# Bloc 3
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

# Bloc 4
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

# Bloc 5
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

# Ajouter des couches supplémentaires
x = layers.GlobalMaxPooling2D(name="global_max_pool")(x)
x = layers.Dropout(0.3, name="dropout")(x)
outputs = layers.Dense(2, activation="softmax", name="predictions")(x)

# Construire le modèle
model = Model(inputs, outputs, name="custom_vgg16.keras")

# Charger les poids de VGG16
vgg16_base = VGG16(include_top=False, weights="imagenet", input_shape=(224, 224, 3))

# Appliquer les poids pour chaque couche manuellement
for layer, pretrained_layer in zip(model.layers, vgg16_base.layers):
    if isinstance(layer, layers.Conv2D):
        layer.set_weights(pretrained_layer.get_weights())


In [14]:
# Figer les poids de VGG16
for layer in model.layers[:-4]:
    layer.trainable = False

# Compile the model

In [15]:
model.compile(
    optimizer = 'adam',
    loss = losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy'])

In [16]:
EPOCHS = 3
history = model.fit(train_batches,
                    epochs = EPOCHS,
                    batch_size = BATCH_SIZE,
                    validation_data = validation_batches)

Epoch 1/3



 1/23 [>.............................] - ETA: 1:34 - loss: 37.2842 - accuracy: 0.3125

KeyboardInterrupt: 

In [ ]:
from compilation import create_model_checkpoint
for layer in model.layers[:-4]:
    layer.trainable = True
# Fine-tune from this layer onwards
fine_tune_at = 2

# Freeze all the layers before the `fine_tune_at` layer
for layer in model.layers[:-fine_tune_at]:
  layer.trainable =  False

model.compile(loss=losses.SparseCategoricalCrossentropy(),
              optimizer = optimizers.Adam(learning_rate=0.0001),
              metrics=['accuracy'])

EPOCHS = 5
history = model.fit(train_batches,
                    epochs = EPOCHS,
                    batch_size=BATCH_SIZE,
                    validation_data = validation_batches,
                    callbacks=[create_model_checkpoint(model_name=model.name)])

In [6]:
# Charger votre modèle
importlib.reload(APoZ)
testt_model = models.load_model("model_experiments/custom_vgg16.keras")

activation_model = Model(
            inputs=model.input,
            outputs=[layer.output for layer in model.layers if isinstance(layer, layers.Conv2D)]
        )
activation_model.predict(train_batches)
activation_model.summary()

 2/23 [=>............................] - ETA: 4:03

KeyboardInterrupt: 

In [7]:
threshold = 0.9
from tests import ActivationAPoZRankedFilterPrune
apoz = APoZ_Algorithm(testt_model, train_batches,threshold)
train_batches

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [8]:
pruned_model = apoz.run()

Running APoZ algorithm...
Analyzing layers for APoZ calculation...
Activation model summary:
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_40 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                

KeyboardInterrupt: 

In [11]:
# pruned_model.summary()